In [21]:

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

from scipy import stats
import time

from sklearn.linear_model import LinearRegression,ElasticNet## 线性回归   弹性网络回归（岭回归和lasso的结合）
from sklearn.svm import SVR## 支持向量回归
from sklearn.kernel_ridge import KernelRidge
import xgboost# GBDT
import lightgbm# GBDT

from sklearn.model_selection import train_test_split

from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.core.models import WeightedEnsembleModel
import warnings
warnings.filterwarnings("ignore")

import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"


In [2]:

data_path = '../data/data_Preliminary_round/'
traindata = pd.read_csv(data_path + 'train.csv')
testdata = pd.read_csv(data_path + 'test.csv')
submit = pd.read_csv('../data/data_Preliminary_round/提交示例.csv')

traindata.info()
## 我要将label值设置为float


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26656 entries, 0 to 26655
Data columns (total 87 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   序号        26656 non-null  int64  
 1   时间        26656 non-null  object 
 2   流量1       26656 non-null  float64
 3   流量2       26656 non-null  float64
 4   流量3       26656 non-null  float64
 5   流量4       26656 non-null  float64
 6   流量5       26656 non-null  float64
 7   流量6       26656 non-null  float64
 8   流量7       26656 non-null  float64
 9   流量8       26656 non-null  float64
 10  流量9       26656 non-null  float64
 11  流量10      26656 non-null  float64
 12  流量11      26656 non-null  float64
 13  流量12      26656 non-null  float64
 14  流量13      26656 non-null  float64
 15  流量14      26656 non-null  float64
 16  流量15      26656 non-null  float64
 17  流量16      26656 non-null  float64
 18  流量17      26656 non-null  float64
 19  上部温度设定1   26656 non-null  int64  
 20  上部温度设定2   26656 non-null  in

In [3]:

traindata.columns = ["Num","Time"]\
+["V"+str(i) for i in range(1,18)]\
+["TSU"+str(i) for i in range(1,18)]\
+["TSD"+str(i) for i in range(1,18)]\
+["TU"+str(i) for i in range(1,18)]\
+["TD"+str(i) for i in range(1,18)]

testdata.columns = ["Num","Time"]\
+["V"+str(i) for i in range(1,18)]\
+["TSU"+str(i) for i in range(1,18)]\
+["TSD"+str(i) for i in range(1,18)]

submit.columns = ["Num"]\
+["TU"+str(i) for i in range(1,18)]\
+["TD"+str(i) for i in range(1,18)]


In [4]:


traindata.drop(["Num","Time"],axis=1,inplace=True)
testdata.drop(["Num","Time"],axis=1,inplace=True)

label = ["TU"+str(i) for i in range(1,18)]\
+["TD"+str(i) for i in range(1,18)]

features = ["V"+str(i) for i in range(1,18)]\
+["TSU"+str(i) for i in range(1,18)]\
+["TSD"+str(i) for i in range(1,18)]


In [5]:

# 尝试一下  不想修改 这边的值类型
# traindata[label] = traindata[label].astype(float)
# traindata.info()



In [6]:

time_limit = 5*60
from tqdm import tqdm
for t in tqdm(label):
    
    predictor = TabularPredictor(label=t,problem_type='regression',eval_metric='mae').fit(traindata[features+[t]], presets='best_quality',time_limit=time_limit)#,time_limit=time_limit
    submit[t] = predictor.predict(testdata)

  0%|          | 0/34 [00:00<?, ?it/s]No path specified. Models will be saved in: "AutogluonModels\ag-20230701_042234\"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 300s
AutoGluon will save models to "AutogluonModels\ag-20230701_042234\"
AutoGluon Version:  0.8.0
Python Version:     3.8.16
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19041
Disk Space Avail:   195.82 GB / 462.26 GB (42.4%)
Train Data Rows:    26656
Train Data Columns: 51
Label Column: TU1
Preprocessing data ...
Using Feature Generators to preprocess the data ...


Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    10369.67 MB
	Train Data (Original)  Memory Usage: 10.88 MB (0.1% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 17 | ['V1', 'V2', 'V3', 'V4', 'V5', ...]
		('int', [])   : 34 | ['TSU1', 'TSU2', 'TSU3', 'TSU4', 'TSU5', ...]
	Types of features in processed data (raw dtype, special dtypes):
		('float', []) : 17 | ['V1', 'V2', 'V3', 'V4', 'V5', ...]
		('int', [])   : 34 | ['TSU1', 'TSU2', 'TSU3', 'TSU4', 'TSU5', ...]
	0.2s = Fit

In [7]:

submit.columns = ['序号','上部温度1', '上部温度2', '上部温度3', '上部温度4', '上部温度5', '上部温度6', '上部温度7',
       '上部温度8', '上部温度9', '上部温度10', '上部温度11', '上部温度12', '上部温度13', '上部温度14',
       '上部温度15', '上部温度16', '上部温度17', '下部温度1', '下部温度2', '下部温度3', '下部温度4',
       '下部温度5', '下部温度6', '下部温度7', '下部温度8', '下部温度9', '下部温度10', '下部温度11',
       '下部温度12', '下部温度13', '下部温度14', '下部温度15', '下部温度16', '下部温度17']

submit.to_csv('submit.csv', index=None)

In [12]:
df_train,df_val = train_test_split(traindata,test_size=0.25,random_state=42)
df_train.shape,df_val.shape

((19992, 85), (6664, 85))

In [ ]:
TabularPredictor

In [19]:
# import os
# for t in os.listdir('./AutogluonModels/'):
#     pre = TabularPredictor.load('./AutogluonModels/'+t)
#     print(pre.label)
    # print(pre.evaluate(df_val[features+[pre.label]]))